In [1]:
import requests
import xml.etree.ElementTree
import socket
import os

## Obtain Token

https://wiki.earthdata.nasa.gov/display/CMR/Creating+a+Token+Common

In [2]:
username = ""
password = ""
url = 'https://cmr.earthdata.nasa.gov/legacy-services/rest/tokens'

In [3]:
def get_ip():
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    try:
        s.connect(('8.8.8.8', 80))
        ip = s.getsockname()[0]
    except:
        ip = '127.0.0.1'
    finally:
        s.close()
    return ip

In [4]:
ip = get_ip()
request = """
<token>
    <username>{username}</username>
    <password>{password}</password>
    <client_id>podpac</client_id>
    <user_ip_address>{ip}</user_ip_address>
</token>
""".format(username=username, password=password,ip=ip)
headers = {"Content-Type": "application/xml"}
r = requests.post(url, data=request, headers=headers)

In [5]:
tree = xml.etree.ElementTree.fromstring(r.text)
token = [element.text for element in tree.findall('id')][0]

In [6]:
token

'CA68AFA4-44BE-6916-C7D6-CE04E7130929'

## Get Data

In [33]:
base_url = "https://n5eil01u.ecs.nsidc.org/egi/request"
# base_url = "https://n5eil02u.ecs.nsidc.org/egi/request"
version = "004"
start_time = "2015-03-30"
end_time = "2015-05-12"
bbox = "-70,42,-69,43"
response_format = "HDF-EOS"
# response_format = "GeoTIFF"
token = token  # from above

product = "SPL4SMAU"
data_layers = "/Analysis_Data/sm_surface_analysis"

# product = "SPL4SMGP"
# data_layers = "/Geophysical_Data/sm_surface"

# product = "SPL3SMP"
# data_layers = "/Soil_Moisture_Retrieval_Data/soil_moisture"

# product = "SPL3SMAP"  # No granule with this name
# data_layers = "/Soil_Moisture_Retrieval_Data/soil_moisture"

# product = "SPL3SMP"
# data_layers = "/Soil_Moisture_Retrieval_Data/soil_moisture"

# product = "SPL3SMP_E"  # No granule with this name
# data_layers = ""


# product = "SPL2SMAP_S"  # No granule with this name
# data_layers = ""


In [36]:
from datetime import datetime
s = datetime.now()
url = "{base_url}?short_name={product}&version={version}".format(base_url=base_url, product=product, version=version,) + \
      "&time={start_time},{end_time}&Coverage={data_layers}".format(start_time=start_time, end_time=end_time, data_layers=data_layers, ) + \
      "&Bbox={bbox}&format={response_format}&token={token}&page_size=20&page_num=50".format(bbox=bbox, response_format=response_format, token=token)

r = requests.get(url)
# r = requests.head(url)
e = datetime.now()
print(e-s)
print(len(r.content))

0:00:00.923364
777


In [30]:
r.headers

{'Date': 'Wed, 10 Jul 2019 14:21:32 GMT', 'Server': 'Apache', 'Connection': 'close', 'Transfer-Encoding': 'chunked', 'Content-Type': 'application/xml'}

In [37]:
r.status_code

501

In [38]:
r.text

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?>\n<eesi:Exception xsi:schemaLocation="http://eosdis.nasa.gov/esi/rsp/e https://newsroom.gsfc.nasa.gov/esi/8.1/schemas/ESIAgentResponseExternal.xsd" xmlns="" xmlns:iesi="http://eosdis.nasa.gov/esi/rsp/i" xmlns:ssw="http://newsroom.gsfc.nasa.gov/esi/rsp/ssw" xmlns:eesi="http://eosdis.nasa.gov/esi/rsp/e" xmlns:esi="http://eosdis.nasa.gov/esi/rsp" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">\n    <Code>InternalError</Code>\n    <Message>No granules returned by CMR: https://cmr.earthdata.nasa.gov/search/granules?provider=NSIDC_ECS&amp;page_num=50&amp;short_name=SPL4SMAU&amp;version=004&amp;temporal=2015-03-30,2015-05-12&amp;token=CA68AFA4-44BE-6916-C7D6-CE04E7130929&amp;page_size=20</Message>\n</eesi:Exception>\n'

In [27]:
r.text

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?>\n<eesi:Exception xsi:schemaLocation="http://eosdis.nasa.gov/esi/rsp/e https://newsroom.gsfc.nasa.gov/esi/8.1/schemas/ESIAgentResponseExternal.xsd" xmlns="" xmlns:iesi="http://eosdis.nasa.gov/esi/rsp/i" xmlns:ssw="http://newsroom.gsfc.nasa.gov/esi/rsp/ssw" xmlns:eesi="http://eosdis.nasa.gov/esi/rsp/e" xmlns:esi="http://eosdis.nasa.gov/esi/rsp" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">\n    <Code>InternalError</Code>\n    <Message>No granules returned by CMR: https://cmr.earthdata.nasa.gov/search/granules?provider=NSIDC_ECS&amp;short_name=SPL4SMAU2&amp;version=004&amp;temporal=2015-03-30,2015-06-12&amp;token=0A1BFF00-2AF1-1801-F91F-EE05AE5E2E4F</Message>\n</eesi:Exception>\n'

## Make manual request

In [20]:
from datetime import datetime
import requests
s = datetime.now()
url = 'https://n5eil01u.ecs.nsidc.org/egi/request?short_name=SPL3SMP_E&format=HDF-EOS&version=002&time=2015-07-06T00:00:00,2015-07-08T00:00:00&Bbox=-82.0,38.0,-81.0,39.0&token=9C23B8B8-C947-57A4-B9E9-836064AB9206'
r = requests.get(url)
e = datetime.now()
print(e-s)

0:01:01.361056


In [19]:
r.text

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?>\n<eesi:Exception xsi:schemaLocation="http://eosdis.nasa.gov/esi/rsp/e https://newsroom.gsfc.nasa.gov/esi/8.1/schemas/ESIAgentResponseExternal.xsd" xmlns="" xmlns:iesi="http://eosdis.nasa.gov/esi/rsp/i" xmlns:ssw="http://newsroom.gsfc.nasa.gov/esi/rsp/ssw" xmlns:eesi="http://eosdis.nasa.gov/esi/rsp/e" xmlns:esi="http://eosdis.nasa.gov/esi/rsp" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">\n    <Code>InternalError</Code>\n    <Message>No granules returned by CMR: https://cmr.earthdata.nasa.gov/search/granules?provider=NSIDC_ECS&amp;short_name=SPL3SMP_E&amp;version=005&amp;temporal=2015-07-06T00:00:00,2015-07-08T00:00:00&amp;token=9C23B8B8-C947-57A4-B9E9-836064AB9206</Message>\n</eesi:Exception>\n'

In [21]:
with open('output.zip', 'wb') as f:
    f.write(r.content)

In [22]:
f = h5py.File('output 2/142675927/SMAP_L3_SM_P_E_20150707_R16020_001_HEGOUT.h5')

In [27]:
[k for k in f['Metadata/Extent'].attrs]

['description',
 'eastBoundLongitude',
 'northBoundLatitude',
 'rangeBeginningDateTime',
 'rangeEndingDateTime',
 'southBoundLatitude',
 'westBoundLongitude']

In [29]:
f['Metadata/Extent'].attrs['rangeEndingDateTime']

b'2015-07-07T23:59:59.999Z'

In [24]:
[k for k in f['Soil_Moisture_Retrieval_Data_PM']]

['EASE_column_index_pm',
 'EASE_row_index_pm',
 'albedo_pm',
 'boresight_incidence_pm',
 'freeze_thaw_fraction_pm',
 'grid_surface_status_pm',
 'latitude_centroid_pm',
 'latitude_pm',
 'longitude_centroid_pm',
 'longitude_pm',
 'radar_water_body_fraction_pm',
 'retrieval_qual_flag_pm',
 'roughness_coefficient_pm',
 'soil_moisture_error_pm',
 'soil_moisture_pm',
 'static_water_body_fraction_pm',
 'surface_flag_pm',
 'surface_temperature_pm',
 'surface_water_fraction_mb_h_pm',
 'surface_water_fraction_mb_v_pm',
 'tb_3_corrected_pm',
 'tb_4_corrected_pm',
 'tb_h_corrected_pm',
 'tb_h_uncorrected_pm',
 'tb_qual_flag_3_pm',
 'tb_qual_flag_4_pm',
 'tb_qual_flag_h_pm',
 'tb_qual_flag_v_pm',
 'tb_time_seconds_pm',
 'tb_time_utc_pm',
 'tb_v_corrected_pm',
 'tb_v_uncorrected_pm',
 'vegetation_opacity_pm',
 'vegetation_water_content_pm']

In [12]:
f = h5py.File('output 8/139973815/SMAP_L4_SM_aup_20150706T000000_Vv4030_001_HEGOUT.h5')

In [13]:
f['x'][()]

array([-7913576.47251037, -7904568.41753112, -7895560.36255187,
       -7886552.30757261, -7877544.25259336, -7868536.19761411,
       -7859528.14263485, -7850520.0876556 , -7841512.03267635,
       -7832503.97769709, -7823495.92271784])

In [14]:
f['y'][()]

array([4607620.47251037, 4598612.41753112, 4589604.36255187,
       4580596.30757261, 4571588.25259336, 4562580.19761411,
       4553572.14263485, 4544564.0876556 , 4535556.03267635,
       4526547.97769709, 4517539.92271784])

## Look at the Data

In [11]:
from podpac.data import Rasterio
import zipfile
from io import BytesIO
%matplotlib inline

In [17]:
# helper function to display all keys in hdf5 file
def explore(f, key):
    print(key)
    print([k for k in f[key]])
    if len(f[key]) and not isinstance(f[key], h5py.Dataset):
        for newkey in f[key]:
            explore(f, '{}/{}'.format(key, newkey))

    if isinstance(f[key], h5py.Dataset):
        print(key + ' attrs:')
        print([k for k in f[key].attrs])

In [56]:
with open('output.zip', 'wb') as f:
    f.write(r.content)

In [ ]:
f = BytesIO(r.content)
zip_file = zipfile.ZipFile(f)

for name in zip_file.namelist():
    h5data = zip_file.read(name)

In [19]:
with zipfile.ZipFile("output.zip","r") as zip_ref:
    zip_ref.extractall()

In [5]:
node = Rasterio(source='test.tiff')

In [6]:
node.native_coordinates

Coordinates
	lat: UniformCoordinates1d(lat): Bounds[4999469.950207469, 5089550.5], N[10], ctype['midpoint']
	lon: UniformCoordinates1d(lon): Bounds[-7143387.5, -7044298.8952282155], N[11], ctype['midpoint']

In [ ]:
o = node.eval(node.native_coordinates)
o.plot()

In [49]:
f = h5py.File('output 6/139973815/SMAP_L4_SM_aup_20150706T000000_Vv4030_001_HEGOUT.h5')

In [50]:
[k for k in f]

['Analysis_Data',
 'Forecast_Data',
 'Metadata',
 'Observations_Data',
 'cell_column',
 'cell_lat',
 'cell_lon',
 'cell_row',
 'time',
 'x',
 'y']

In [53]:
f = h5py.File('output 4/139973815/SMAP_L4_SM_aup_20150706T000000_Vv4030_001_HEGOUT.h5')

In [54]:
[k for k in f]

['Forecast_Data', 'Metadata', 'projection_information', 'time', 'x', 'y']

In [29]:
f = h5py.File('output 3/81805640/SMAP_L3_SM_AP_20150707_R13080_001_HEGOUT.h5')

## PODPAC Interface

In [1]:
from podpac.datalib import EGI
from podpac import settings

## EGI

In [ ]:
username = ""
password = ""
token = "DCF3DD10-C56C-1F7F-12FE-6408839955B8"

In [ ]:
node = EGI(token=token, short_name="SPL3SMA", version="003", data_key="/Soil_Moisture_Retrieval_Data/soil_moisture")

In [ ]:
node

In [ ]:
node.token_valid()

In [ ]:
node.get_token()

In [ ]:
node.token_valid()

In [ ]:
node.token

## SMAP

In [1]:
import podpac
from podpac.datalib.smap_egi import SMAP
import zipfile
import h5py

from podpac import Coordinates, clinspace
import numpy as np

import logging
logger = logging.getLogger('podpac')
logger.setLevel(logging.DEBUG)

In [2]:
c = Coordinates([clinspace(-82, -81.5, 10), clinspace(38, 38.5, 10), clinspace('2015-07-06', '2015-07-07', 10)], dims=['lon', 'lat', 'time'])
node = SMAP()

In [3]:
%pdb
o = node.eval(c)

Automatic pdb calling has been turned ON


DEBUG:podpac.datalib.egi:EGI Token valid
DEBUG:podpac.datalib.egi:Querying EGI url: https://n5eil01u.ecs.nsidc.org/egi/request?short_name=SPL4SMAU&Coverage=/Analysis_Data/sm_surface_analysis,/x,/y&format=HDF-EOS&version=004&time=2015-07-06T00:00:00,2015-07-07T00:00:00&Bbox=-82.0,38.0,-81.5,38.5&token=8F2245D7-998A-14EF-8406-FFE4D9EDDA06&page_size=2&page_num=1
DEBUG:podpac.datalib.egi:Querying EGI url: https://n5eil01u.ecs.nsidc.org/egi/request?short_name=SPL4SMAU&Coverage=/Analysis_Data/sm_surface_analysis,/x,/y&format=HDF-EOS&version=004&time=2015-07-06T00:00:00,2015-07-07T00:00:00&Bbox=-82.0,38.0,-81.5,38.5&token=8F2245D7-998A-14EF-8406-FFE4D9EDDA06&page_size=2&page_num=2
DEBUG:podpac.datalib.egi:Querying EGI url: https://n5eil01u.ecs.nsidc.org/egi/request?short_name=SPL4SMAU&Coverage=/Analysis_Data/sm_surface_analysis,/x,/y&format=HDF-EOS&version=004&time=2015-07-06T00:00:00,2015-07-07T00:00:00&Bbox=-82.0,38.0,-81.5,38.5&token=8F2245D7-998A-14EF-8406-FFE4D9EDDA06&page_size=2&page_nu

In [4]:
o

<xarray.UnitsDataArray (lon: 10, lat: 10, time: 10)>
array([[[     nan,      nan, ...,      nan,      nan],
        [     nan,      nan, ...,      nan,      nan],
        ...,
        [     nan,      nan, ...,      nan,      nan],
        [     nan,      nan, ...,      nan,      nan]],

       [[     nan,      nan, ...,      nan,      nan],
        [     nan,      nan, ...,      nan,      nan],
        ...,
        [     nan,      nan, ...,      nan,      nan],
        [     nan,      nan, ...,      nan,      nan]],

       ...,

       [[0.226789, 0.231432, ..., 0.213835, 0.213835],
        [0.22779 , 0.231924, ..., 0.213679, 0.213679],
        ...,
        [0.294267, 0.292974, ..., 0.274226, 0.274226],
        [     nan,      nan, ...,      nan,      nan]],

       [[0.227755, 0.231684, ..., 0.213755, 0.213755],
        [0.231722, 0.235287, ..., 0.216737, 0.216737],
        ...,
        [0.300178, 0.299188, ..., 0.281802, 0.281802],
        [     nan,      nan, ...,      nan,      na

In [6]:
node.native_coordinates

Coordinates (epsg:6933)
	time: ArrayCoordinates1d(time): Bounds[2015-07-05T13:00:00.000000000, 2015-07-05T16:00:00.000000000], N[2], ctype['point']
	lon: ArrayCoordinates1d(lon): Bounds[4517539.697614107, 4562579.972510373], N[6], ctype['midpoint']
	lat: ArrayCoordinates1d(lat): Bounds[-7913576.472510373, -7868536.197614107], N[6], ctype['midpoint']

In [5]:
a = node.native_coordinates.transform('epsg:4326')

/Users/marcus/computing/creare/podpac/podpac/podpac/core/coordinates/dependent_coordinates.py:491: UserWarning: transformation of coordinate segment lengths not yet implemented
  warnings.warn("transformation of coordinate segment lengths not yet implemented")
/Users/marcus/computing/creare/podpac/podpac/podpac/core/coordinates/dependent_coordinates.py:493: UserWarning: transformation of coordinate segment lengths not yet implemented
  warnings.warn("transformation of coordinate segment lengths not yet implemented")


In [8]:
a['lat'].coordinates

array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]])

In [8]:
node.native_coordinates.transform(crs="epsg:4326")

/Users/marcus/computing/creare/podpac/podpac/podpac/core/coordinates/dependent_coordinates.py:491: UserWarning: transformation of coordinate segment lengths not yet implemented
  warnings.warn("transformation of coordinate segment lengths not yet implemented")
/Users/marcus/computing/creare/podpac/podpac/podpac/core/coordinates/dependent_coordinates.py:493: UserWarning: transformation of coordinate segment lengths not yet implemented
  warnings.warn("transformation of coordinate segment lengths not yet implemented")
/anaconda3/envs/podpac/lib/python3.7/site-packages/numpy/core/fromnumeric.py:83: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Coordinates(epsg:4326)
	time: ArrayCoordinates1d(time): Bounds[2015-07-05T13:00:00.000000000, 2015-07-06T16:00:00.000000000], N[10], ctype['point']
	lat,lon[lat]: DependentCoordinates(i,j->lat): Bounds[nan, nan], shape(11, 11), ctype[midpoint]
	lat,lon[lon]: DependentCoordinates(i,j->lon): Bounds[46.820542, 47.754152], shape(11, 11), ctype[midpoint]